In [7]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
import datetime
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from plotly.subplots import make_subplots
import plotly.graph_objs as go

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
folder = "../00_Raw_Data/"

# Generate filenames from 2015_Q1 to 2022_Q4
years = range(2015, 2023)
quarters = range(1, 5)

files = ["{}_Q{}_Traffic_Crashes.csv".format(year, quarter) for year in years for quarter in quarters]

dfs = [pd.read_csv(folder + file) for file in files]
df = pd.concat(dfs, ignore_index=True)

In [3]:
df['collision_date'] = pd.to_datetime(df['collision_date'], format='%m/%d/%Y %I:%M:%S %p')
df['date'] = df['collision_date'].dt.date

df['day_of_week'] = df['collision_date'].dt.dayofweek # 0: Monday - 6: Sunday
df['weekend'] = df['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)

df['time'] = df['collision_date'].dt.time
df['hour'] = df['collision_date'].dt.hour
df['day_of_month'] = df['collision_date'].dt.day
df['month'] = df['collision_date'].dt.month
df['year'] = df['collision_date'].dt.year
df['covid'] = (df['date'] >= pd.Timestamp('2020-03-15').date()).astype(int)

columns_to_convert = ['pedestrian', 'bicycle', 'scooter', 'hitrun', 'parking']

for column in columns_to_convert:
    df[column] = df[column].map({'yes': 1, 'no': 0})
    
    
df['manner_of_crash'] = df['manner_of_crash'].fillna('Unknown/Missing')
df['manner_of_crash'] = df['manner_of_crash'].replace(['Missing', 'Unknown'], 'Unknown/Missing')
crash_onehot = pd.get_dummies(df['manner_of_crash'], prefix='crash')
df = pd.concat([df, crash_onehot], axis=1)
df.drop('manner_of_crash', axis=1, inplace=True)

df.drop('roadway_name', axis=1, inplace=True)

df['intersect_type'].replace(['OTHER', 'Missing', 'Unknown'], 'Unknown/Other', inplace=True)
intersect_type_onehot = pd.get_dummies(df['intersect_type'], prefix='intersect_type')
df = pd.concat([df, intersect_type_onehot], axis=1)
df.drop('intersect_type', axis=1, inplace=True)

df['relation_to_junction'].replace(['Missing', 'OtherLocation', 'Unknown'], 'Unknown/Other', inplace=True)
relation_onehot = pd.get_dummies(df['relation_to_junction'], prefix='relation_to_junction')
df = pd.concat([df, relation_onehot], axis=1)
df.drop('relation_to_junction', axis=1, inplace=True)

df.drop('city', axis=1, inplace=True)

df['intersection_indicator'] = df['intersection_indicator'].fillna("N")
df['intersection_indicator'].replace("Missing", "N", inplace=True)
mapping = {"N": 0, "Y": 1}
df['intersection_indicator'] = df['intersection_indicator'].map(mapping)
df['intersection_indicator'] = df['intersection_indicator'].astype('int64')

# Combine "Campus" and "OHO" categories
df['mou'] = df['mou'].replace('OHO', 'Campus')

# Now, encode "Campus" as 1 and "False" as 0
df['mou'] = df['mou'].apply(lambda x: 1 if x == 'Campus' else 0)

df['work_zone_type'] = df['work_zone_type'].apply(lambda x: 0 if x == "Missing" or x == "Unknown" or x is None else 1)


def simplify_weather(weather):
    if pd.isna(weather):
        return 'Unknown/Other'
    elif 'Snow' in weather:
        return 'Snow'
    elif 'Sleet/Hail' in weather:
        return 'Sleet/Hail'
    elif 'Rain' in weather:
        return 'Rain'
    elif weather in ['Fog', 'Smoke', 'Smog']:
        return 'Fog'
    elif 'Cloudy' in weather:
        return 'Cloudy'
    elif weather == 'Clear':
        return 'Clear'
    elif weather in ['Missing', 'Unknown', 'Other']:
        return 'Unknown/Other'
    else:
        return 'Other Conditions'

df['simplified_weather'] = df['weather_condition(s)'].apply(simplify_weather)
df = pd.get_dummies(df, columns=['simplified_weather'], prefix='weather')
df.drop('weather_condition(s)', axis=1, inplace=True)

def bool_to_int(df):
    for col in df.select_dtypes(['bool']).columns:
        df[col] = df[col].astype(int)
    return df

df = bool_to_int(df)

# df['time'] = df['time'].astype(str)
# df['time'] = df['time'].str.strip()
# has_time_df = df[df['time'] != "00:00:00"]

In [10]:
fig = make_subplots(rows=7, cols=1, subplot_titles=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

for day in range(7):
    filtered_df = df[df['day_of_week'] == day]
    hour_counts = filtered_df['hour'].value_counts().sort_index()
    fig.add_trace(
        go.Bar(x=hour_counts.index, y=hour_counts.values, name=f'Day {day}'),
        row=day + 1, col=1
    )
fig.update_layout(height=1400, width=600, title_text="Accidents by Hour for Each Day of the Week")
fig.show()


In [11]:
df['time'] = df['time'].astype(str)
df['time'] = df['time'].str.strip()
df = df[df['time'] != "00:00:00"]

In [20]:
weekday_df = df[df['day_of_week'].between(0, 4)].copy()  

def categorize_time_window(hour):
    if 6 <= hour <= 9:
        return 'rush_morning'
    elif 15 <= hour <= 18:
        return 'rush_evening'
    elif 10 <= hour <= 14:
        return 'non_rush_day'
    else:  
        return 'non_rush_night'

weekday_df['time_window'] = weekday_df['hour'].apply(categorize_time_window)

In [22]:
time_window_counts = weekday_df['time_window'].value_counts()
total = time_window_counts.sum()
time_window_percentages = (time_window_counts / total) * 100

fig = px.bar(time_window_counts, x=time_window_counts.index, y=time_window_counts.values,
             labels={'y': 'Count', 'x': 'Time Window'},
             text=[f"{pct:.2f}%" for pct in time_window_percentages.values])

# Update hover data to show both count and percentage
fig.update_traces(hovertemplate='%{x}: %{y} rows<br>%{text}')

# Update layout and titles
fig.update_layout(title_text='Count and Percentage for Each Time Window in Weekday Data')
fig.show()